In [ ]:
import os
import os.path as osp
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
meta_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/frame_records_csvs'
verify_video_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/verify_videos'

driver_ver_dct ={
    'atalie': 1,
    'keli': 1,
    'matt': 1,
    'subject5_cap': 2,
    'subject5_tied_hair': 2,
    'subject25_cap': 2,
    'subject25_tied_hair': 2,
}

def get_image_list(dn, ver):
    if ver == 1:
        image_list_record_root = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data_wrap4d/Pair_man/PAC/meta/train'
    elif ver == 2:
        image_list_record_root = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/meta'

    meta_file = osp.join(image_list_record_root, f'data_parsed_List_{dn}_man_process_list_train.csv')
    if ver == 1:
        image_filename_list = pd.read_csv(meta_file)['tar_bg'].tolist()
    elif ver == 2:
        image_filename_list = pd.read_csv(meta_file)['tar'].tolist()
    return image_filename_list